In [24]:
using Polynomials
push!(LOAD_PATH, pwd())
using Controlz
using Test
using DifferentialEquations
using Logging
using DataFrames
using LaTeXStrings
using PyPlot
PyPlot.matplotlib.style.use("hipster.mplstyle")

In [ ]:
##
# second order, underdamped
K = 4.3
τ = 2.8
ξ = 0.2
g = K / (τ ^ 2 * s ^ 2 + 2 * τ * ξ * s + 1)
# impulse response
t, y = simulate(s/s^2, (0.0, 25.0))

viz_response(t, y, plot_title="underdamped step response")

In [ ]:
# test transfer function algebra (P-control regulator step response)
τ = 1.4
K = 2.8
Kc = 3.3
Kd = 4.5
τd = 8.2
gd = Kd / (τd * s + 1)
gcgp = K * Kc / (τ * s + 1)
y_ovr_d = gd / (1 + gcgp)
@test isapprox(y_ovr_d, TransferFunction(Kd/(1+Kc*K) * [τ, 1], [τd*τ/(1+Kc*K), (τ+τd+Kc*K*τd)/(1+Kc*K), 1]))

In [ ]:
viz_poles_and_zeros(g)

In [ ]:
viz_nyquist_diagram(g)

In [ ]:
g_ol = 4 / (s + 3) / (s + 2) / (s + 1)

viz_root_locus(g_ol)